In [1]:
from proxmox.docs import _proxmox_api_docs
from langchain.text_splitter import RecursiveJsonSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from typing import Dict, Any
from langchain_community.llms import Ollama
from proxmox.base import ProxmoxAPIChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from tqdm import tqdm, trange  
import os
import hvac


C:\Users\oumay\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
splitter = RecursiveJsonSplitter(max_chunk_size=4000)
docs = splitter.create_documents(_proxmox_api_docs)

In [3]:
llm_kwargs: Dict[str, Any] = {
    "model": "llama3.1:instruct",
    "temperature": 0,
    "num_ctx": 4096, 
}

llm = Ollama(**llm_kwargs)

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize the llm with the new model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0)

In [5]:
def get_vault_token():
    token = os.getenv('VAULT_TOKEN')
    if not token:
        token = input("Vault token: ").strip()
        os.environ['VAULT_TOKEN'] = token
    return token

# Function to access the secret from Vault
def access_vault_secret(vault_token, url, secret_path):
    try:
        client = hvac.Client(url=url, token=vault_token)
        response = client.secrets.kv.v2.read_secret_version(path=secret_path, mount_point="kv")
        if response:
            secret_data = response['data']['data']
            print(f"Accessed Vault secret at {secret_path}.")
            return secret_data
        else:
            print(f"Failed to access Vault secret at {secret_path}.")
            return None
    except hvac.exceptions.VaultError as e:
        print(f"Request to Vault failed: {e}")
        return None

# Get Vault token from environment or prompt
vault_token = os.getenv('VAULT_TOKEN') or get_vault_token()

# Save Vault token in environment variables if not already set
os.environ['VAULT_TOKEN'] = vault_token

# Retrieve Vault URL from environment variables
vault_url = os.getenv('VAULT_URL')

# Paths to secrets in Vault
proxmox_secret_path = "users/accounts/api-tokens/proxmox/oumaima"
surrealdb_url_path = "users/accounts/surrealdb"
surrealdb_credentials_path = "users/accounts/surrealdb/oumaima"

# Access Proxmox API token from Vault
proxmox_api_token = access_vault_secret(vault_token, vault_url, proxmox_secret_path)
if proxmox_api_token:
    os.environ['PROXMOX_API_TOKEN'] = proxmox_api_token.get('proxmox-api-token')
else:
    print("Failed to retrieve Proxmox API token.")

# Access SurrealDB URL from Vault
surrealdb_url_secret = access_vault_secret(vault_token, vault_url, surrealdb_url_path)
if surrealdb_url_secret:
    os.environ['SURREALDB_URL'] = surrealdb_url_secret.get('url')
else:
    print("Failed to retrieve SurrealDB URL.")

# Access SurrealDB credentials from Vault
surrealdb_credentials_secret = access_vault_secret(vault_token, vault_url, surrealdb_credentials_path)
if surrealdb_credentials_secret:
    os.environ['SURREALDB_USER'] = surrealdb_credentials_secret.get('user')
    os.environ['SURREALDB_PWD'] = surrealdb_credentials_secret.get('password')
else:
    print("Failed to retrieve SurrealDB credentials.")


C:\Users\oumay\AppData\Local\Temp\ipykernel_9784\2293483164.py:12: DeprecationWarning: The raise_on_deleted_version parameter will change its default value to False in hvac v3.0.0. The current default of True will presere previous behavior. To use the old behavior with no warning, explicitly set this value to True. See https://github.com/hvac/hvac/pull/907
  response = client.secrets.kv.v2.read_secret_version(path=secret_path, mount_point="kv")


Accessed Vault secret at users/accounts/api-tokens/proxmox/oumaima.
Accessed Vault secret at users/accounts/surrealdb.
Accessed Vault secret at users/accounts/surrealdb/oumaima.


In [6]:
proxmox_api_token = os.getenv('PROXMOX_API_TOKEN')
surrealdb_url = os.getenv('SURREALDB_URL')
surrealdb_user = os.getenv('SURREALDB_USER')
surrealdb_password = os.getenv('SURREALDB_PWD')

In [7]:
# Initialize the SentenceTransformerEmbeddings with trust_remote_code=True
ef = SentenceTransformerEmbeddings(
    model_name="infgrad/stella_en_400M_v5",
    model_kwargs={
        "trust_remote_code": True,
    }
)

c:\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Some weights of the model checkpoint at infgrad/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassif

In [8]:
import nest_asyncio
from surrealdb import Surreal

nest_asyncio.apply()

# SurrealDB connection parameters
dburl = "ws://localhost:8000/rpc"
ns = "langchain"
db_name = "proxmox_api_docs"
collection = "proxmox_collection"
db_user = surrealdb_user
db_pass = surrealdb_password

In [9]:
from langchain_community.vectorstores.surrealdb import SurrealDBStore

# Initialize the SurrealDBStore with documents
db = SurrealDBStore(
    embedding_function=ef,
    dburl=dburl,
    ns=ns,
    db=db_name,
    collection=collection,
    db_user=db_user,
    db_pass=db_pass
)

In [11]:
# this is needed to initialize the underlying async library for SurrealDB
await db.initialize()

# delete collection from the vectorstore collection if it exists 
await db.adelete(collection = collection)

# add documents to the vectorstore
ids = await db.aadd_documents(docs)

In [12]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [13]:
# Initialize the cross-encoder model
#cross_encoder_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
cross_encoder_model = HuggingFaceCrossEncoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")
compressor = CrossEncoderReranker(model=cross_encoder_model, top_n=3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [15]:
chain = ProxmoxAPIChain.from_llm_and_api_docs(
    llm,
    retriever=compression_retriever,
    headers={"Authorization": proxmox_api_token},
    verbose=True,
    limit_to_domains=['https://ns31418912.ip-54-38-37.eu:8006'],
    base_url= 'https://ns31418912.ip-54-38-37.eu:8006/',
)

c:\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


# Vms management: 

## LiST available Vms on a proxmoxs node :

In [ ]:
user_input = (
    f"list Vms on node 'Proxmox-Node-HCM'"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...


c:\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
C:\Users\oumay\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu", "method": "get", "summary": "List Virtual machines (VMs) on a Node", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where VMs are located", "schema": {"type": "string"}}]}
Retrieved Document 1: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu/{vmid}/config", "method": "get", "summary": "Get Virtual machine/Vm Configuration (config file)", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "ID of the VM", "schema": {"type": "integer"}}, {"name": "current", "in": "query", "required": false, "description": "Get current values instead of pending values", "schema": {"type": "boolean"}}]}
Retrieved Document 2: {"base_url": "http

## Creating a new Vm : 

### Vm with initial config :

In [ ]:
user_input = (
    f"Create a VM on node 'Proxmox-Node-HCM' with the following configuration :\n"
    f"VM ID: 10000\n"
    f"Name: test-vm-3\n"
    f"Memory: 512 MB\n"
    f"Cores: 1\n"
    f"Sockets: 1\n"
    f"Storage: local\n"
    f"ide0: local:32,format=qcow2\n"
    f"net0: virtio,bridge=vmbr0\n"
    f"OS Type: l26\n"
    f"Description: My test VM\n"
    f"NUMA: 1\n"
    f"KVM: 1\n"
    f"Agent: 1"
)

response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu", "method": "post", "summary": "Create a Virtual machine (VM) ", "description": "Use this endpoint to create a new virtual machine on the specified node. This endpoint is used for initial creation and can be used for the allocation of resources such as memory, cores, and storage.", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM will be created", "schema": {"type": "string"}}], "requestBody": {"required": true, "content": {"application/json": {"schema": {"type": "object", "properties": {"vmid": {"type": "integer", "description": "VM ID"}, "name": {"type": "string", "description": "Name of the VM"}, "memory": {"type": "integer", "description": "Memory size in MB"}, "cores": {"type": "integer", "description": "Number of CPU cores"}, "ide0": {"type": "string", "description": "D

### Vm with just the required fields : 

In [ ]:
user_input = (
    f"Create a Vm on node 'Proxmox-Node-HCM' where Vm ID : 2020"
)

response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu", "method": "post", "summary": "Create a Virtual machine (VM) ", "description": "Use this endpoint to create a new virtual machine on the specified node. This endpoint is used for initial creation and can be used for the allocation of resources such as memory, cores, and storage.", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM will be created", "schema": {"type": "string"}}], "requestBody": {"required": true, "content": {"application/json": {"schema": {"type": "object", "properties": {"vmid": {"type": "integer", "description": "VM ID"}, "name": {"type": "string", "description": "Name of the VM"}, "memory": {"type": "integer", "description": "Memory size in MB"}, "cores": {"type": "integer", "description": "Number of CPU cores"}, "ide0": {"type": "string", "description": "D

## Manage Vm Config : 

### Update Vm config :

In [ ]:
user_input = (
    f"adjust the configuration of the VM with id 10000 on node 'Proxmox-Node-HCM' with memory 2048 MiB, 2 cores, CPU type 'host', name 'test-vm', operating system 'win10', and boot order 'order=ide0'"
)
response = chain.invoke(user_input)

print(response['output'])




> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu/{vmid}/config", "method": "put", "summary": "Update virtual machine configuration options", "description": "Use this endpoint to modify the configuration of an existing virtual machine. This is useful when you need to change settings such as memory, cores, network devices, and other VM parameters for an already running or stopped VM.", "parameters": [{"name": "node", "in": "path", "required": true, "description": "The cluster node name.", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "The (unique) ID of the VM.", "schema": {"type": "integer", "minimum": 100, "maximum": 999999999}}], "requestBody": {"required": true, "content": {"application/json": {"schema": {"type": "object", "properties": {"acpi": {"type": "boolean", "description": "Enable/disable ACPI."}, "agent": {"type": "s

### Set initial config :

In [ ]:
user_input = (
    f"set the initial configuration of the VM with id 2020 on node 'Proxmox-Node-HCM', memory: 512, cputype: kvm64, name: test-vm-2020"
)
response = chain.invoke(user_input)

print(response['output'])




> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu/{vmid}/config", "method": "post", "summary": "Set initial configuration options of a Virtual machine", "description": "Use this endpoint to set the initial configuration of a new virtual machine. This is useful when you need to define settings such as memory, cores, CPU type, and other parameters for a VM that is being created.", "parameters": [{"name": "node", "in": "path", "required": true, "description": "The cluster node name", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "The (unique) ID of the VM", "schema": {"type": "integer", "minimum": 100, "maximum": 999999999}}], "requestBody": {"required": true, "content": {"application/json": {"schema": {"type": "object", "properties": {"memory": {"type": "integer", "description": "Amount of RAM for the VM in MiB", "minimum": 16}, 

### Display Config :

In [ ]:
user_input = (
    f"get the configuration of the Vm with id 10000 on node 'Proxmox-Node-HCM'"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu/{vmid}/config", "method": "get", "summary": "Get Virtual machine/Vm Configuration (config file)", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "ID of the VM", "schema": {"type": "integer"}}, {"name": "current", "in": "query", "required": false, "description": "Get current values instead of pending values", "schema": {"type": "boolean"}}]}
Retrieved Document 1: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu", "method": "get", "summary": "List Virtual machines (VMs) on a Node", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where VMs are located", "schema": {"type": "string"}}]

In [ ]:
user_input = (
    f"get the configuration of the Vm with id 2020 on node 'Proxmox-Node-HCM'"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu/{vmid}/config", "method": "get", "summary": "Get Virtual machine/Vm Configuration (config file)", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "ID of the VM", "schema": {"type": "integer"}}, {"name": "current", "in": "query", "required": false, "description": "Get current values instead of pending values", "schema": {"type": "boolean"}}]}
Retrieved Document 1: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc/{vmid}/config", "method": "get", "summary": "Get Lxc container configuration/config ", "parameters": [{"name": "node", "in": "path", "required": true, "description": "The cluster node name", "schema": {"type": "str

## Delete Vm :

In [ ]:
user_input = (
    f"delete Vm with id 10000 on node 'Proxmox-Node-HCM'"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu/{vmid}", "method": "delete", "summary": "Delete Virtual machine (VM)", "description": "Deleting a Virtual machine from a node .", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "ID of the VM to delete", "schema": {"type": "string"}}]}
Retrieved Document 1: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc/{vmid}", "methods": "delete", "summary": "Delete LXC Container", "description": "Deleting a container from a node .", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the LXC container is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true,

In [ ]:
user_input = (
    f"delete Vm with id 2020 on node 'Proxmox-Node-HCM'"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu/{vmid}", "method": "delete", "summary": "Delete Virtual machine (VM)", "description": "Deleting a Virtual machine from a node .", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "ID of the VM to delete", "schema": {"type": "string"}}]}
Retrieved Document 1: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc/{vmid}", "methods": "delete", "summary": "Delete LXC Container", "description": "Deleting a container from a node .", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the LXC container is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true,

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 16.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Request info: https://ns31418912.ip-54-38-37.eu:8006/api2/json/nodes/Proxmox-Node-HCM/qemu/2020 | DELETE | {} 

API URL: https://ns31418912.ip-54-38-37.eu:8006/api2/json/nodes/Proxmox-Node-HCM/qemu/2020
Request method: DELETE
Request body: {}
Status: 200 OK
Response: {"data":"UPID:Proxmox-Node-HCM:001F466C:AFA101B0:669E2A06:qmdestroy:2020:oumaima@pve!oumaima-token:"}

> Finished chain.
The API successfully deleted the VM with ID 2020 on the node 'Proxmox-Node-HCM'. 



# Containers Management :

## List all available containers :

In [ ]:
user_input = (
    f"list all containers on node 'Proxmox-Node-HCM'"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc", "methods": "get", "summary": "List LXC Containers on a Node", "description": "List all containers available on a node", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where LXC containers are located", "schema": {"type": "string"}}]}
Retrieved Document 1: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc/{vmid}/config", "method": "put", "summary": "Set container configuration (config file)", "description": "Modify or set the configuration of an LXC container identified by {vmid} on the specified {node}.", "parameters": [{"name": "node", "in": "path", "required": true, "description": "The cluster node name", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "The (unique) ID of the VM", "schema": 

## Create Container :

### New container with initial config :

In [ ]:
user_input = (
    "Create a container on node 'Proxmox-Node-HCM' where Container ID : 1010\n"
    "Hostname: my-container\n"
    "Memory (in MB): 512\n"
    "CPU Cores: 1\n"
    "Root Filesystem (e.g., 'local:10' for 10GB): local:10\n"
    "Network Configuration (e.g., 'name=eth0,bridge=vmbr0'): name=eth0,bridge=vmbr0\n"
    "OS Template: local:vztmpl/ubuntu-22.04-standard_22.04-1_amd64.tar.zst"
)

response = chain.invoke(user_input)

print(response['output'])




> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc", "methods": "post", "summary": "Create LXC Container", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the LXC container will be created", "schema": {"type": "string"}}], "requestBody": {"required": true, "content": {"application/json": {"schema": {"type": "object", "properties": {"vmid": {"type": "integer", "description": "Container ID"}, "hostname": {"type": "string", "description": "Hostname of the container"}, "memory": {"type": "integer", "description": "Memory size in MB"}, "cores": {"type": "integer", "description": "Number of CPU cores"}, "rootfs": {"type": "string", "description": "Root filesystem, e.g., 'local:10' for 10GB"}, "net0": {"type": "string", "description": "Network configuration, e.g., 'bridge=vmbr0'"}, "ostemplate": {"type": "string", "description": "OS templa

### New container with just the required fields :

In [ ]:
user_input = (
    "Create a container on node 'Proxmox-Node-HCM' where Container ID : 3030\n"
    "OS Template: local:vztmpl/ubuntu-22.04-standard_22.04-1_amd64.tar.zst"
)

response = chain.invoke(user_input)

print(response['output'])




> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc", "methods": "post", "summary": "Create LXC Container", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the LXC container will be created", "schema": {"type": "string"}}], "requestBody": {"required": true, "content": {"application/json": {"schema": {"type": "object", "properties": {"vmid": {"type": "integer", "description": "Container ID"}, "hostname": {"type": "string", "description": "Hostname of the container"}, "memory": {"type": "integer", "description": "Memory size in MB"}, "cores": {"type": "integer", "description": "Number of CPU cores"}, "rootfs": {"type": "string", "description": "Root filesystem, e.g., 'local:10' for 10GB"}, "net0": {"type": "string", "description": "Network configuration, e.g., 'bridge=vmbr0'"}, "ostemplate": {"type": "string", "description": "OS templa

## Manage container's config :

### Update Config :

In [ ]:
user_input = (
    f"modify the configuration of the container with id 1010 on node 'Proxmox-Node-HCM', Hostname: mycontainer-2"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc/{vmid}/config", "method": "put", "summary": "Set container configuration (config file)", "description": "Modify or set the configuration of an LXC container identified by {vmid} on the specified {node}.", "parameters": [{"name": "node", "in": "path", "required": true, "description": "The cluster node name", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "The (unique) ID of the VM", "schema": {"type": "integer"}}], "requestBody": {"required": true, "content": {"application/json": {"schema": {"type": "object", "properties": {"arch": {"type": "string", "enum": ["amd64", "i386", "arm64", "armhf", "riscv32", "riscv64"], "description": "OS architecture type"}, "cmode": {"type": "string", "enum": ["shell", "console", "tty"], "description": "Console mode"}, "console": {"type": "boolean", 

### Get Config :

In [ ]:
user_input = (
    f"get the configuration of the container with id 1010 on node 'Proxmox-Node-HCM'"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc/{vmid}/config", "method": "get", "summary": "Get Lxc container configuration/config ", "parameters": [{"name": "node", "in": "path", "required": true, "description": "The cluster node name", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "The (unique) ID of the VM", "schema": {"type": "integer"}}, {"name": "current", "in": "query", "required": false, "description": "Get current values (instead of pending values)", "schema": {"type": "boolean", "default": false}}, {"name": "snapshot", "in": "query", "required": false, "description": "Fetch config values from given snapshot", "schema": {"type": "string"}}]}
Retrieved Document 1: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc/{vmid}/config", "method": "put", "summary": "Set container 

## Delete Container :

In [ ]:
user_input = (
    f"delete Container with id 1010 on node 'Proxmox-Node-HCM'"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc/{vmid}", "methods": "delete", "summary": "Delete LXC Container", "description": "Deleting a container from a node .", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the LXC container is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "ID of the LXC container to delete", "schema": {"type": "string"}}]}
Retrieved Document 1: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu/{vmid}", "method": "delete", "summary": "Delete Virtual machine (VM)", "description": "Deleting a Virtual machine from a node .", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "requi

In [ ]:
user_input = (
    f"delete Container with id 3030 on node 'Proxmox-Node-HCM'"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/lxc/{vmid}", "methods": "delete", "summary": "Delete LXC Container", "description": "Deleting a container from a node .", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the LXC container is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "required": true, "description": "ID of the LXC container to delete", "schema": {"type": "string"}}]}
Retrieved Document 1: {"base_url": "https://ns31418912.ip-54-38-37.eu:8006/", "endpoint": "/api2/json/nodes/{node}/qemu/{vmid}", "method": "delete", "summary": "Delete Virtual machine (VM)", "description": "Deleting a Virtual machine from a node .", "parameters": [{"name": "node", "in": "path", "required": true, "description": "Node ID where the VM is located", "schema": {"type": "string"}}, {"name": "vmid", "in": "path", "requi

# Get all users :

In [ ]:
user_input = (
    f"get all users"
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"endpoint": "/api2/json/access/permissions", "method": "get", "summary": "Retrieve/get effective permissions of given user/token.", "parameters": [{"name": "path", "in": "query", "required": false, "description": "Only dump this specific path, not the whole tree.", "schema": {"type": "string"}}, {"name": "userid", "in": "query", "required": false, "description": "User ID or full API token ID", "schema": {"type": "string"}}]}
Retrieved Document 1: {"endpoint": "/api2/json/access/users", "method": "get", "summary": "User index.", "parameters": [{"name": "enabled", "in": "query", "required": false, "description": "Optional filter for enable property.", "schema": {"type": "boolean"}}, {"name": "full", "in": "query", "required": false, "description": "Include group and token information.", "schema": {"type": "boolean", "default": false}}]}
Retrieved Document 2: {"endpoint": "/api2/json/nodes/{node}/qemu", "method": "get", "sum

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..



> Finished chain.
The API response lists all users. 



# Get user permissions :

In [ ]:
user_input = (
    f"get permissions of user with id oumaima@pve "
)
response = chain.invoke(user_input)

print(response['output'])



> Entering new ProxmoxAPIChain chain...
Retrieved Document 0: {"endpoint": "/api2/json/access/permissions", "method": "get", "summary": "Retrieve/get effective permissions of given user/token.", "parameters": [{"name": "path", "in": "query", "required": false, "description": "Only dump this specific path, not the whole tree.", "schema": {"type": "string"}}, {"name": "userid", "in": "query", "required": false, "description": "User ID or full API token ID", "schema": {"type": "string"}}]}
Retrieved Document 1: {"endpoint": "/api2/json/access/users", "method": "get", "summary": "User index.", "parameters": [{"name": "enabled", "in": "query", "required": false, "description": "Optional filter for enable property.", "schema": {"type": "boolean"}}, {"name": "full", "in": "query", "required": false, "description": "Include group and token information.", "schema": {"type": "boolean", "default": false}}]}
Retrieved Document 2: {"endpoint": "/api2/json/nodes/{node}/qemu/{vmid}/config", "method